In [1]:
import json
import pyttsx3


class FinanceManager:
    def __init__(self):
        self.balance = 0
        self.categories = {}
        self.expenses = {}

        # Initialize text-to-speech engine
        self.speech_engine = pyttsx3.init()
        self.speech_engine.setProperty('rate', 180)  # Set speaking rate
        self.speech_engine.setProperty('volume', 1)  # Set volume level (0.0 to 1.0)

    def speak(self, message):
        """Convert text to speech."""
        print(f"Assistant says: {message}")
        self.speech_engine.say(message)
        self.speech_engine.runAndWait()

    def set_balance(self):
        """Set or update the user's total balance."""
        self.speak("Enter the amount you want to add to your budget in PKR.")
        balance_text = input("Enter the amount to add to your budget: PKR ")
        new_balance = float(balance_text)
        self.balance += new_balance
        self.speak(f"Budget updated successfully. Total balance is now PKR {self.balance:.2f}.")
        print(f"Budget updated successfully. Total balance is now: PKR {self.balance:.2f}")

    def add_categories_with_priority(self):
        """Add spending categories with priorities."""
        self.speak("Enter the number of categories you'd like to add.")
        num_categories = int(input("Enter the number of categories: "))
        for _ in range(num_categories):
            self.speak("Enter category name.")
            category = input("Enter category name: ")
            self.speak(f"Enter priority for category '{category}' (1 is most important, 5 is least).")
            priority = int(input(f"Enter priority for '{category}' (1-5): "))
            self.categories[category] = {"priority": priority, "allocation": 0}
        self.speak("Categories added successfully with priorities.")
        print("\nCategories added with priorities.")

    def suggest_allocation(self):
        """Suggest budget allocation based on priorities."""
        self.speak("What percentage of your budget would you like to allocate automatically? (Enter a number between 0 and 100).")
        percentage_text = input("Enter percentage to allocate (0-100): ")
        try:
            percentage_to_allocate = float(percentage_text)
            if not (0 <= percentage_to_allocate <= 100):
                raise ValueError("Percentage must be between 0 and 100.")
        except ValueError as e:
            self.speak("Invalid input. Please enter a valid percentage.")
            print("Error:", e)
            return

        # Temporary variable for allocation budget (does not change the main balance)
        allocation_budget = (percentage_to_allocate / 100) * self.balance
        total_priority = sum(1 / details["priority"] for details in self.categories.values())

        temp_allocations = {}
        for category, details in self.categories.items():
            allocation_percentage = (1 / details["priority"]) / total_priority
            allocated_amount = allocation_budget * allocation_percentage
            temp_allocations[category] = allocated_amount
            self.speak(f"Suggested allocation for {category}: PKR {allocated_amount:.2f} (Priority {details['priority']})")
            print(f"Suggested allocation for {category}: PKR {allocated_amount:.2f} (Priority {details['priority']})")
            self.speak(f"Suggested allocation completed.")
            print(f"Suggested allocation completed.")



    def manual_allocation(self):
        """Allow the user to manually allocate the budget to categories."""
        total_allocated = sum(details["allocation"] for details in self.categories.values())
        total_spent = sum(details["spent"] for details in self.expenses.values())
        remaining_budget = self.balance - total_allocated - total_spent

        if remaining_budget <= 0:
            self.speak("No remaining budget available for allocation. Please update your budget first.")
            print("No remaining budget available for allocation. Please update your budget first.")
            return

        self.speak(f"You have PKR {remaining_budget:.2f} available for allocation.")
        print(f"Remaining budget: PKR {remaining_budget:.2f}")

        for category in self.categories:
            while True:
                self.speak(f"Enter amount for category '{category}' (leave 0 to skip). Remaining budget: PKR {remaining_budget:.2f}")
                amount = float(input(f"Enter amount for '{category}' (leave 0 to skip): PKR "))
                if amount > remaining_budget:
                    self.speak("Warning! You are exceeding the remaining budget. Please enter a lower amount.")
                    print("Warning! You are exceeding the remaining budget.")
                else:
                    self.categories[category]["allocation"] += amount
                    self.expenses[category] = self.expenses.get(category, {"allocated": 0, "spent": 0})
                    self.expenses[category]["allocated"] += amount
                    remaining_budget -= amount
                    break

        self.speak(f"Manual allocation completed. Remaining budget: PKR {remaining_budget:.2f}")
        print(f"Manual allocation completed. Remaining budget: PKR {remaining_budget:.2f}")

    def add_expense(self):
        """Add an expense to a category."""
        while True:
            self.speak("Enter the category to add expense.")
            category = input("Enter category to add expense: ")
            if category not in self.expenses:
                self.speak(f"Category '{category}' does not exist. Please try again.")
                print(f"Category '{category}' does not exist.")
                continue

            self.speak(f"Enter the expense amount for category '{category}'.")
            amount = float(input("Enter expense amount: PKR "))
            if amount > self.expenses[category]["allocated"] - self.expenses[category]["spent"]:
                self.speak(f"Insufficient funds in '{category}'! Try a smaller amount.")
                print(f"Insufficient funds in '{category}'! Try a smaller amount.")
                continue
            self.expenses[category]["spent"] += amount
            self.speak(f"PKR {amount:.2f} spent in '{category}'. Remaining: PKR {self.expenses[category]['allocated'] - self.expenses[category]['spent']:.2f}")
            print(f"PKR {amount:.2f} spent in '{category}'. Remaining: PKR {self.expenses[category]['allocated'] - self.expenses[category]['spent']:.2f}")
            break

    def view_summary(self):
        """View financial summary."""
        self.speak("Viewing financial summary.")
        total_spent = 0
        for category, details in self.expenses.items():
            total_spent += details["spent"]
            self.speak(f"{category}: Allocated = PKR {details['allocated']:.2f}, Spent = PKR {details['spent']:.2f}, Remaining = PKR {details['allocated'] - details['spent']:.2f}")
            print(f"{category}: Allocated = PKR {details['allocated']:.2f}, Spent = PKR {details['spent']:.2f}, Remaining = PKR {details['allocated'] - details['spent']:.2f}")
        self.speak(f"Total Balance: PKR {self.balance:.2f}, Total Spent: PKR {total_spent:.2f}, Remaining: PKR {self.balance - total_spent:.2f}")
        print(f"Total Balance: PKR {self.balance:.2f}, Total Spent: PKR {total_spent:.2f}, Remaining: PKR {self.balance - total_spent:.2f}")

    def view_savings(self):
        """View remaining savings."""
        total_spent = sum(details["spent"] for details in self.expenses.values())
        savings = self.balance - total_spent
        self.speak(f"Your remaining savings: PKR {savings:.2f}")
        print(f"Remaining Savings: PKR {savings:.2f}")

    def save_data(self, filename="finance_data.json"):
        """Save financial data to a file."""
        data = {
            "balance": self.balance,
            "categories": self.categories,
            "expenses": self.expenses
        }
        with open(filename, "w") as file:
            json.dump(data, file)
        self.speak(f"Data saved to file.")
        print(f"Data saved to file")

    def view_saved_data(self, filename="finance_data.json"):
        """View saved data from the JSON file."""
        try:
            with open(filename, "r") as file:
                data = json.load(file)
                print("\n--- Saved Data ---")
                print(json.dumps(data, indent=4))  # Pretty print the JSON data
                self.speak("Here is the saved data.")
        except FileNotFoundError:
            self.speak("No saved data found.")
            print("No saved data found.")


# Main Program
manager = FinanceManager()

manager.speak("Hello, I am your personal finance manager.\n My name is Pennywise, helping you become smart with every penny.\n I am here to assist you.")

while True:
    # Display options
    manager.speak("Please enter your choice.")
    print("\n--- Finance Manager ---")
    print("1. Set Balance")
    print("2. Add Categories with Priorities")
    print("3. Suggest Budget Allocation")
    print("4. Manual Allocation")
    print("5. Add Expense")
    print("6. View Summary")
    print("7. View Savings")
    print("8. Save Data")
    print("9. View Saved Data")
    print("0. Exit")

    choice = input("Enter your choice (0-10): ")

    if choice == "1":
        manager.set_balance()
    elif choice == "2":
        manager.add_categories_with_priority()
    elif choice == "3":
        manager.suggest_allocation()
    elif choice == "4":
        manager.manual_allocation()
    elif choice == "5":
        manager.add_expense()
    elif choice == "6":
        manager.view_summary()
    elif choice == "7":
        manager.view_savings()
    elif choice == "8":
        manager.save_data()
    elif choice == "9":
        manager.view_saved_data()
    elif choice == "0":
        manager.speak("Goodbye!")
        print("Exiting Finance Manager. Goodbye!")
        break
    else:
        manager.speak("Invalid choice. Please try again.")
        print("Invalid choice. Please try again.")


Assistant says: Hello, I am your personal finance manager.
 My name is Pennywise, helping you become smart with every penny.
 I am here to assist you.
Assistant says: Please enter your choice.

--- Finance Manager ---
1. Set Balance
2. Add Categories with Priorities
3. Suggest Budget Allocation
4. Manual Allocation
5. Add Expense
6. View Summary
7. View Savings
8. Save Data
9. View Saved Data
0. Exit


Enter your choice (0-10):  1


Assistant says: Enter the amount you want to add to your budget in PKR.


Enter the amount to add to your budget: PKR  50000


Assistant says: Budget updated successfully. Total balance is now PKR 50000.00.
Budget updated successfully. Total balance is now: PKR 50000.00
Assistant says: Please enter your choice.

--- Finance Manager ---
1. Set Balance
2. Add Categories with Priorities
3. Suggest Budget Allocation
4. Manual Allocation
5. Add Expense
6. View Summary
7. View Savings
8. Save Data
9. View Saved Data
0. Exit


...

Assistant says: Viewing financial summary.
Assistant says: bill: Allocated = PKR 10000.00, Spent = PKR 0.00, Remaining = PKR 10000.00
bill: Allocated = PKR 10000.00, Spent = PKR 0.00, Remaining = PKR 10000.00
Assistant says: rent: Allocated = PKR 20000.00, Spent = PKR 0.00, Remaining = PKR 20000.00
rent: Allocated = PKR 20000.00, Spent = PKR 0.00, Remaining = PKR 20000.00
Assistant says: shopping: Allocated = PKR 10000.00, Spent = PKR 10000.00, Remaining = PKR 0.00
shopping: Allocated = PKR 10000.00, Spent = PKR 10000.00, Remaining = PKR 0.00
Assistant says: Total Balance: PKR 50000.00, Total Spent: PKR 10000.00, Remaining: PKR 40000.00
Total Balance: PKR 50000.00, Total Spent: PKR 10000.00, Remaining: PKR 40000.00
Assistant says: Please enter your choice.

--- Finance Manager ---
1. Set Balance
2. Add Categories with Priorities
3. Suggest Budget Allocation
4. Manual Allocation
5. Add Expense
6. View Summary
7. View Savings
8. Save Data
9. View Saved Data
0. Exit


Enter your choice (0-10):  7


Assistant says: Your remaining savings: PKR 40000.00
Remaining Savings: PKR 40000.00
Assistant says: Please enter your choice.

--- Finance Manager ---
1. Set Balance
2. Add Categories with Priorities
3. Suggest Budget Allocation
4. Manual Allocation
5. Add Expense
6. View Summary
7. View Savings
8. Save Data
9. View Saved Data
0. Exit


Enter your choice (0-10):  0


Assistant says: Goodbye!
Exiting Finance Manager. Goodbye!
